In [3]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.

<ipython-input-3-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-3-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, 0 to 560
dtypes: float64(561)
memory usage: 31.5 MB


In [6]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
df=st.fit_transform(df)
df

array([[ 0.20064157, -0.0636826 , -0.41962845, ..., -0.68721921,
         0.40794614, -0.00756789],
       [ 0.05594788,  0.03148567, -0.25390836, ..., -0.694138  ,
         0.40911698,  0.00787517],
       [ 0.07351535, -0.04341648, -0.07629468, ..., -0.702239  ,
         0.4102883 ,  0.02650234],
       ...,
       [-0.01566765,  0.0167814 ,  1.13222107, ..., -0.56584847,
         0.64059683,  0.34870928],
       [ 0.21586648, -0.02812252, -0.86770988, ..., -0.57766781,
         0.63147758,  0.29327564],
       [ 1.09620157,  0.12919873, -1.67268082, ..., -0.57392691,
         0.63274259,  0.33396081]])

In [7]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df,y,test_size=0.25,random_state=1)
xtrain

array([[ 2.25982767,  0.13646002,  0.11571968, ..., -0.19792178,
         0.92243948,  0.83135764],
       [ 0.04411624, -0.2277251 ,  0.19175243, ..., -0.8711634 ,
         0.05333659,  0.41980263],
       [ 0.04192655,  0.02572733, -0.02100604, ..., -0.68293517,
         0.44876846,  0.04359406],
       ...,
       [ 0.02640686, -0.02025411,  0.04242439, ..., -0.32376084,
         0.80382605,  0.73905251],
       [ 0.03942031,  0.08444794, -0.0541105 , ..., -0.47755521,
        -0.1868364 , -0.32803986],
       [-0.21800208,  0.0244102 ,  0.221871  , ...,  1.83759575,
        -2.57910916, -0.7687313 ]])

In [8]:
# SOLUTION -  K-Means for dimensionality reduction
n_clusters = 50
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df.T)
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df[:, selected_features_indices]

In [10]:
df=selected_features

In [11]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df,y,test_size=0.25,random_state=1)
xtrain

array([[ 0.04735575,  1.19475628,  1.04514681, ..., -0.07676241,
        -0.52428864,  0.18056461],
       [-0.23889251, -0.97355102, -0.78014913, ...,  0.69003593,
        -0.0884327 , -0.48485441],
       [ 0.55422333, -0.97355102, -0.78214974, ..., -0.82316801,
         1.07689567,  0.18056461],
       ...,
       [-0.01689411, -0.97355102, -0.7829029 , ..., -0.63716052,
         1.39234935, -0.70666073],
       [-0.83800588, -0.97355102, -0.78140768, ...,  1.05469018,
         0.57850346,  0.18056461],
       [ 0.77022388, -0.67343349, -0.78130245, ...,  1.63303973,
        -1.0888989 ,  0.18056461]])

In [12]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
ypred=model.fit(xtrain,ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
ypred=model.predict(xtest)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,ypred)

0.8220892274211099